In [6]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="Bkn7HMszGVliBmChzz78"
)

result = CLIENT.infer("test3.jpg", model_id="corrobot/2")
print(result)

{'inference_id': '938df2a1-6702-4670-a65a-7602cad6063d', 'time': 0.029469064998920658, 'image': {'width': 640, 'height': 640}, 'predictions': [{'x': 431.0, 'y': 270.0, 'width': 80.0, 'height': 64.0, 'confidence': 0.8757827281951904, 'class': 'corrosion', 'points': [{'x': 422.0, 'y': 239.0}, {'x': 418.0, 'y': 243.0}, {'x': 417.0, 'y': 243.0}, {'x': 416.0, 'y': 244.0}, {'x': 414.0, 'y': 244.0}, {'x': 413.0, 'y': 245.0}, {'x': 412.0, 'y': 245.0}, {'x': 410.0, 'y': 247.0}, {'x': 409.0, 'y': 247.0}, {'x': 407.0, 'y': 249.0}, {'x': 406.0, 'y': 249.0}, {'x': 403.0, 'y': 252.0}, {'x': 401.0, 'y': 252.0}, {'x': 397.0, 'y': 256.0}, {'x': 396.0, 'y': 256.0}, {'x': 395.0, 'y': 257.0}, {'x': 394.0, 'y': 257.0}, {'x': 392.0, 'y': 259.0}, {'x': 392.0, 'y': 276.0}, {'x': 397.0, 'y': 281.0}, {'x': 397.0, 'y': 282.0}, {'x': 400.0, 'y': 285.0}, {'x': 400.0, 'y': 286.0}, {'x': 401.0, 'y': 287.0}, {'x': 401.0, 'y': 288.0}, {'x': 404.0, 'y': 291.0}, {'x': 404.0, 'y': 293.0}, {'x': 406.0, 'y': 295.0}, {'x': 

In [9]:
import cv2
import numpy as np

# 元画像を読み込む
img = cv2.imread("test3.jpg")

# オーバーレイ用の空画像を用意（元画像と同じサイズ）
overlay = img.copy()

# マスク＋ラベルを描画
for pred in result['predictions']:
    points = pred['points']
    label = pred['class']
    confidence = pred['confidence']

    # ポリゴン用に点を整形
    polygon = np.array([[int(p['x']), int(p['y'])] for p in points], dtype=np.int32)
    polygon = polygon.reshape((-1, 1, 2))  # OpenCVの要求形式

    # マスク色
    mask_color = (0, 255, 255)  # 水色（BGR）
    
    # ポリゴンを塗りつぶし
    cv2.fillPoly(overlay, [polygon], mask_color)

# 透明度をかけて合成
alpha = 0.4
img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

# ラベル（クラス名＋信頼度）を表示
for pred in result['predictions']:
    points = pred['points']
    label = pred['class']
    confidence = pred['confidence']

    # 中心を計算してラベルを置く
    xs = [p['x'] for p in points]
    ys = [p['y'] for p in points]
    center_x = int(sum(xs) / len(xs))
    center_y = int(sum(ys) / len(ys))

    text = f"{label} {int(confidence * 100)}%"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    font_thickness = 2

    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_x = center_x - text_size[0] // 2
    text_y = center_y + text_size[1] // 2

    # ラベル背景（マスク色と合わせる）
    cv2.rectangle(img, (text_x, text_y - text_size[1]), (text_x + text_size[0], text_y), (0, 255, 255), -1)
    # ラベル文字（黒文字）
    cv2.putText(img, text, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness)

# 保存
cv2.imwrite("output_polygon_masked.jpg", img)

print("ポリゴンマスク＋ラベル付きで保存しました！")


ポリゴンマスク＋ラベル付きで保存しました！
